# Lab 09 · Agent Memory with SQLite

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A SQLite schema is designed for sessions, messages, and memories.
- A memory summarization plan is drafted for periodic runs.
- Data access helpers are introduced.

## What will be learned
- SQLite migrations are sketched for conversational data.
- Summarization planning is discussed for memory consolidation.
- Repository patterns for data access are reinforced.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install sqlite-utils
```

## Step-by-step tasks
### Step 1: Schema file
A schema file is provided to capture sessions, messages, and memories.

In [ ]:
from pathlib import Path
schema_path = Path("ai-web/backend/app/schema.sql")
schema_path.write_text('''CREATE TABLE IF NOT EXISTS sessions (
  id TEXT PRIMARY KEY,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS messages (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  session_id TEXT REFERENCES sessions(id),
  role TEXT,
  content TEXT,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS memories (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  session_id TEXT REFERENCES sessions(id),
  summary TEXT,
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  next_review TIMESTAMP
);
''')
print("Schema file was produced.")

### Step 2: Repository helper
A helper module is included for interacting with SQLite.

In [ ]:
from pathlib import Path
repo_path = Path("ai-web/backend/app/repository.py")
repo_path.write_text('''import sqlite3
from pathlib import Path
from typing import Iterable

DB_PATH = Path(__file__).resolve().parent / "data.sqlite3"


def get_connection():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn


def apply_schema(schema_sql: str):
    conn = get_connection()
    conn.executescript(schema_sql)
    conn.commit()
    conn.close()


def insert_message(session_id: str, role: str, content: str):
    conn = get_connection()
    conn.execute(
        "INSERT INTO messages (session_id, role, content) VALUES (?, ?, ?)",
        (session_id, role, content),
    )
    conn.commit()
    conn.close()


def list_recent_messages(session_id: str, limit: int = 10) -> Iterable[sqlite3.Row]:
    conn = get_connection()
    rows = conn.execute(
        "SELECT role, content FROM messages WHERE session_id = ? ORDER BY created_at DESC LIMIT ?",
        (session_id, limit),
    ).fetchall()
    conn.close()
    return rows
''')
print("Repository helper was created.")

### Step 3: Summarization plan
A plan is described to summarize conversations periodically.

A periodic job is proposed in which conversations are summarized into the memories table. The job is scheduled after a session surpasses a message threshold. A placeholder function is positioned so later labs can hook in summarization calls.

In [ ]:
from pathlib import Path
plan_path = Path("ai-web/backend/app/memory_plan.py")
plan_path.write_text('''from datetime import datetime, timedelta


def plan_memory_summary(session_id: str) -> dict:
    return {
        "session_id": session_id,
        "next_review": (datetime.utcnow() + timedelta(hours=6)).isoformat(),
        "status": "A summarization run has been scheduled.",
    }
''')
print("Memory plan placeholder was drafted.")

## Validation / acceptance checks
```bash
# locally
curl http://localhost:8000/health
```
- The backend health endpoint remains operational after SQLite helpers are introduced.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- A cron-style schedule is documented for invoking the summarization plan.
- Additional indexing strategies are explored for the messages table.